Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
df['created'] = pd.to_datetime(df['created'])

In [8]:
df.sample(5)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
9067,1.0,1,2016-04-13 01:27:33,**NEW on the market** This extra large 1-bedro...,West 57th Street,40.7686,-73.9867,2500,425 West 57th Street,low,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14200,2.0,4,2016-06-22 03:25:06,LOCATION: 180th &amp; Broadway.*Call Corey any...,West 180th Street,40.8495,-73.9360,3400,661 West 180th Street,low,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18755,1.0,1,2016-06-09 13:24:59,"986 Jefferson\r1 bed\r1 bath\r$2,200\r\rDishwa...",986 Jefferson Ave,40.6869,-73.9182,2200,986 Jefferson Ave,low,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32948,1.0,0,2016-05-25 05:30:47,This Beautifully designed Building boasts exqu...,West 39th Street,40.7556,-73.9922,3000,330 West 39th Street,low,1,0,1,0,1,1,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
10735,1.0,3,2016-06-18 13:46:13,Gorgeous Riverside Dr Location! Washer/Dryer I...,Riverside Dr & 161st Street,40.8374,-73.9465,2945,884 Riverside Drive,medium,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
df['created'].dtype


dtype('<M8[ns]')

In [10]:
type(df['created'])

pandas.core.series.Series

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48817 entries, 0 to 49351
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   bathrooms             48817 non-null  float64       
 1   bedrooms              48817 non-null  int64         
 2   created               48817 non-null  datetime64[ns]
 3   description           47392 non-null  object        
 4   display_address       48684 non-null  object        
 5   latitude              48817 non-null  float64       
 6   longitude             48817 non-null  float64       
 7   price                 48817 non-null  int64         
 8   street_address        48807 non-null  object        
 9   interest_level        48817 non-null  object        
 10  elevator              48817 non-null  int64         
 11  cats_allowed          48817 non-null  int64         
 12  hardwood_floors       48817 non-null  int64         
 13  dogs_allowed    

In [12]:
#Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [53]:
df2016 = df[df['created'].dt.year==2016]

In [59]:
df2016.shape

(48817, 34)

In [54]:
train = df2016[df2016['created'].dt.month==5&4]

In [55]:
train.shape

(16217, 34)

In [57]:
test = df[df['created'].dt.month==6]

In [58]:
test.shape

(16973, 34)

In [60]:
#Engineer at least two new features. (See below for explanation & ideas.)

In [61]:
train.sample(10)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
1175,1.0,0,2016-04-02 02:44:58,"Actual Pics!!!A BEAUTIFUL BUILDING, LOCATED IN...",Astor Place,40.7301,-73.9927,3195,1 Astor Place,low,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
7747,1.0,0,2016-04-17 03:13:42,I have an access to show several thousand avai...,W 34 Street,40.7546,-73.9983,2200,455 W 34 Street,low,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3185,1.0,1,2016-04-15 04:12:54,Spacious Large One Bedroom With L-Shap...,Selfridge St,40.7134,-73.8570,1700,68-20 Selfridge St,low,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3697,1.0,1,2016-04-26 02:52:49,Murray Hill apartment now available to rent. T...,Park Avenue,40.7490,-73.9797,3200,49 Park Avenue,low,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48707,1.0,1,2016-04-12 05:26:17,**Just listed and Priced to rent quick**- Bea...,York Avenue,40.7802,-73.9444,3140,1775 York Avenue,low,1,0,1,0,1,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
190,1.0,2,2016-04-27 01:23:59,This apartment was recently renovated with an ...,East 60th Street,40.7616,-73.9643,4600,245 East 60th Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4703,1.0,1,2016-04-06 07:50:42,"BEAUTIFUL STUDIO IN TRIBECA W/ DOORMAN, 24 HOU...",Worth Street,40.7163,-74.0042,3295,95 Worth Street,medium,1,1,1,1,1,1,1,0,1,0,1,1,1,1,0,1,0,0,0,0,1,0,0,0
5470,1.0,2,2016-04-07 06:09:50,If you're looking for an amazing 2 bedroom apa...,Grove Street,40.7334,-74.0018,4495,92 Grove Street,medium,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0
48012,1.5,2,2016-04-20 05:33:26,Tired of that endless search for that spacious...,West 73rd Street,40.7776,-73.9777,4450,36 West 73rd Street,low,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0
4903,1.0,2,2016-04-08 03:39:37,Location: Arden St &amp; Nagle AveCALL: LIAM (...,Arden Street,40.8614,-73.9266,1885,1 Arden Street,low,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
train.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                  object
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [72]:
train['yuppy1']=np.where(train['loft']==1,1,0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
train['yuppy2']=np.where(train['roof_deck']==1,1,0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
train.shape

(16217, 36)

In [78]:
train['yuppy2'].value_counts()

0    14074
1     2143
Name: yuppy2, dtype: int64

In [86]:
test['yuppy1']=np.where(test['loft']==1,1,0)
test['yuppy2']=np.where(test['roof_deck']==1,1,0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
#Fit a linear regression model with at least two features.

In [83]:
#Use scikit-learn for linear regression with one feature.
from sklearn.linear_model import LinearRegression

In [84]:
model = LinearRegression()

In [85]:
features = ['latitude','longitude']
target = ['price']

In [88]:
x_train = train[features]
y_train = train[target]

In [89]:
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [94]:

x_test = test[features]
y_pred = model.predict(x_test)
y_pred

array([[2975.83325274],
       [3540.31579781],
       [3218.287736  ],
       ...,
       [3878.21079305],
       [3603.25789104],
       [3621.40047732]])

In [91]:
# Get the model's coefficients and intercept

In [95]:
print(model.coef_)
print(model.intercept_)

[[  2216.40652711 -15980.44386783]]
[-1268898.02099232]


In [97]:
from sklearn.metrics import mean_absolute_error

In [98]:
mae = mean_absolute_error(y_train, y_pred)

ValueError: ignored